In [1]:
import numpy as np
import pandas as pd
from glob import glob
import cv2
import tensorflow as tf
from tensorflow import keras

In [10]:
import json
with open('DeepRuleDataset/bardata(1031)/bar/annotations/instancesBar(1031)_val2019.json') as f:
    data = json.load(f)

In [11]:
data.keys()

dict_keys(['licenses', 'images', 'annotations', 'categories'])

In [12]:
data['images'][0:5]

[{'file_name': 'c49285ca77f6aff6214dad492b688113_d3d3LmRhbmUuZ292LmNvCTE3MC4yMzguNjQuNzg=.xls-0-0.png',
  'height': 426,
  'width': 1027,
  'id': 6262},
 {'file_name': 'c49285ca77f6aff6214dad492b688113_d3d3LmRhbmUuZ292LmNvCTE3MC4yMzguNjQuNzg=.xls-0-1.png',
  'height': 427,
  'width': 1028,
  'id': 6263},
 {'file_name': 'c49830551627d9220fc08c5e9fe007b6_d3d3Lmdvdi5zY290CTEzNC4xOS4xNjEuMjQ5.xls-25-0.png',
  'height': 381,
  'width': 1008,
  'id': 6264},
 {'file_name': 'c49bfcb997437ebb0afc13986b889817_d3d3LndpbmVwaS5jb20JMjA3LjE1MC4yMTIuOTk=.xls-0-0.png',
  'height': 562,
  'width': 964,
  'id': 6265},
 {'file_name': 'c49bfcb997437ebb0afc13986b889817_d3d3LndpbmVwaS5jb20JMjA3LjE1MC4yMTIuOTk=.xls-1-0.png',
  'height': 561,
  'width': 964,
  'id': 6266}]

In [13]:
data['annotations'][0:5]

[{'image_id': 6262,
  'category_id': 0,
  'bbox': [110.0, 70.0, 17.0, 223.0],
  'area': 3791.0,
  'id': 143084},
 {'image_id': 6262,
  'category_id': 0,
  'bbox': [152.0, 65.0, 17.0, 228.0],
  'area': 3876.0,
  'id': 143085},
 {'image_id': 6262,
  'category_id': 0,
  'bbox': [194.0, 82.0, 17.0, 211.0],
  'area': 3587.0,
  'id': 143086},
 {'image_id': 6262,
  'category_id': 0,
  'bbox': [235.0, 67.0, 17.0, 226.0],
  'area': 3842.0,
  'id': 143087},
 {'image_id': 6262,
  'category_id': 0,
  'bbox': [277.0, 54.0, 17.0, 239.0],
  'area': 4063.0,
  'id': 143088}]

In [14]:
len(data['images'])

6121

In [15]:
len(data['annotations'])

139108

In [22]:
data['categories']

[{'supercategory': 'Series', 'id': 0, 'name': 'Series'}]

In [23]:
# Sample lists of dictionaries
list1 = [{'id': 1, 'name': 'A'}, {'id': 2, 'name': 'B'}, {'id': 3, 'name': 'C'}]
list2 = [{'id': 2, 'details': 'info1'}, {'id': 1, 'details': 'info2'}, {'id': 2, 'details': 'info3'}, {'id': 3, 'details': 'info4'}, {'id': 1, 'details': 'info5'}]

# Create a dictionary from list2 for faster lookup
dict2 = {item['id']: item for item in list2}

# Match elements from list1 with elements from list2
matched_elements = [{**item1, **item2} for item1 in list1 for item2 in filter(lambda x: x['id'] == item1['id'], list2)]

# Print the matched elements
print(matched_elements)


[{'id': 1, 'name': 'A', 'details': 'info2'}, {'id': 1, 'name': 'A', 'details': 'info5'}, {'id': 2, 'name': 'B', 'details': 'info1'}, {'id': 2, 'name': 'B', 'details': 'info3'}, {'id': 3, 'name': 'C', 'details': 'info4'}]


In [2]:
import hourglass
from hourglass import StackedHourglassNetwork

In [3]:
help(StackedHourglassNetwork)

Help on function StackedHourglassNetwork in module hourglass:

StackedHourglassNetwork(input_shape=(256, 256, 3), num_stack=4, num_residual=1, num_heatmap=16)



In [11]:
hg = StackedHourglassNetwork(input_shape=(256, 256, 1), num_stack=1)
hg.summary()

Model: "stacked_hourglass"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_868 (Conv2D) │ (None, 128, 128,  │      3,200 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_868[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_820 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ re_lu_820[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_821 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_870 (Conv2D) │ (None, 128, 128,  │      4,160 │ re_lu_821[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_870[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_822 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_871 (Conv2D) │ (None, 128, 128,  │     36,928 │ re_lu_822[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_871[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_823 (ReLU)    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_869 (Conv2D) │ (None, 128, 128,  │      8,320 │ re_lu_820[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_872 (Conv2D) │ (None, 128, 128,  │      8,320 │ re_lu_823[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_335 (Add)       │ (None, 128, 128,  │          0 │ conv2d_869[0][0], │
│                     │ 128)              │            │ conv2d_872[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_61    │ (None, 64, 64,    │          0 │ add_335[0][0]     │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ max_pooling2d_61

 Total params: 4,299,024 (16.40 MB)

 Trainable params: 4,278,288 (16.32 MB)

 Non-trainable params: 20,736 (81.00 KB)

In [12]:
import cv2
img = cv2.imread('DeepRuleDataset/bardata(1031)/bar/images/train2019/0a0a66e55c96ffd21f3e362e3fbbc816_Y29udGVudC5kaWdpdGFsLm5ocy51awkxOTQuMTg5LjI3LjI4-1-0.png')

In [13]:
img.shape

(321, 328, 3)

In [15]:
# Check if the image was successfully loaded
if img is not None:
    # Display the image
    cv2.imshow('Image', img)
    cv2.waitKey(0)  # Wait for any key press to close the window
    cv2.destroyAllWindows()  # Close all OpenCV windows
else:
    print("Image not found or could not be loaded.")
